In [1]:
"""
Optuna example that optimizes a Keras model
"""
import optuna
from optuna.integration.mlflow import MLflowCallback

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import utils
import tensorflow as tf


print(f"TensorFlow {tf.__version__}")
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    print(f"Number of available GPUs : {len(gpus)}")
    tf.config.set_visible_devices(gpus[0],"GPU")
    tf.config.experimental.set_memory_growth(gpus[0],True)
else:
    print("No GPU available, using CPU !!!")    

# To disable GPU use
tf.config.set_visible_devices([], 'GPU')
print("Not using GPU !!!")    


NCONSTIT=32
NFEAT=3
NTRIALS=50
METRIC='categorical_accuracy'
NEPOCHS = 50
BSIZES = [32,64,128]  # batch size for Optuna search
MINLR = 0.0001
MAXLR = 0.0003
MAXNL = 3 # maximum number of layer for Optuna search
MINNEU = 8 # minimum number of neurons per layer for Optuna search
MAXNEU = 256 # maximum number of neurons per layer for Optuna search

# OPTUNA trial syntax
# filters=trial.suggest_categorical("filters", [32, 64]),
# activation=trial.suggest_categorical("activation", ["relu", "linear"]),
# learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
# num_hidden = trial.suggest_int("n_units_l{}".format(i), 4, 128, log=True)            
            

def create_model(trial):
    model = Sequential()

    # Define the MLP.
    nlayers = trial.suggest_int("nlayers", 1, MAXNL)
    model = Sequential()
    model.add(Input(shape=(NCONSTIT*NFEAT)))
    for i in range(nlayers):
        nhidden = trial.suggest_int("nhidden_l{}".format(i), MINNEU, MAXNEU, log=True)
        model.add(Dense(nhidden, activation="relu"))
    model.add(Dense(5, activation="softmax"))

    
    # Define the optimizer
    optim = Adam( learning_rate= trial.suggest_float("learning_rate", MINLR, MAXLR, log=True) )

# compile the model
    model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=[METRIC])
    return model


def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()
    
    # early stopping callback
    es = EarlyStopping(monitor=METRIC, patience=10)

    # Learning rate scheduler 
    ls = ReduceLROnPlateau(monitor=METRIC, factor=0.2, patience=10)


    # model checkpoint callback
    # this saves our model architecture + parameters into bestmodel.h5
    chkp = ModelCheckpoint('bestmodel.h5', monitor=METRIC, verbose=0, save_best_only=True, mode='auto', save_freq='epoch')

#
    model = create_model(trial)
    model.fit(X_train, Y_train, 
              batch_size= trial.suggest_categorical("bsize", BSIZES),  
              epochs=NEPOCHS, 
              verbose=0,
              validation_data=(X_val, Y_val),
              callbacks=[es,ls,chkp],
              shuffle=True,
              )
    # Load the best model
    model = tf.keras.models.load_model( 'bestmodel.h5',)
    
#                                         custom_objects={ "QDense": QDense,"QActivation": QActivation,
#                                                          "QConv1D": QConv1D, "quantized_bits": quantized_bits,
#                                                          "NodeEdgeProjection": NodeEdgeProjection,} )                                        
    
    
    # Evaluate the model on the validation data
    score = model.evaluate(X_test, Y_test, verbose=2)

    return score[1] # return the model accuracy



TensorFlow 2.8.0
Number of available GPUs : 1
Not using GPU !!!


In [2]:
import util.util
import util.data
#from . import util as dsutil


'''
# Load data 
X_test = np.ascontiguousarray( np.load("{}/x_test_{}const.npy".format(datapath, nconst)) )
Y_test = np.load( "{}/y_test_{}const.npy".format(datapath, nconst), allow_pickle=True )
'''



util.util.device_info()
outdir = "./optuna"


kfolds = 5
val_kfold = 4

print("kfold numbet of folds:", kfolds )
print("val fold num:", val_kfold )

train_kfolds = [kfold for kfold in range(kfolds) if kfold != val_kfold]

fpath        =  f'./data_kfold/jets_{NCONSTIT}constituents_ptetaphi_nonorm'
fnames_train = [f'jet_images_c{NCONSTIT}_minpt2.0_ptetaphi_nonorm_{train_kfold}' for train_kfold in train_kfolds]
fname_val    =  f'jet_images_c{NCONSTIT}_minpt2.0_ptetaphi_nonorm_{val_kfold}'


data = util.data.Data.load_kfolds(fpath, fnames_train, fname_val)
print(data.train_data.shape)

X_train = data.train_data
X_val = data.test_data
X_test = data.test_data

Y_train = data.train_target
Y_val = data.test_target
Y_test = data.test_target

# normalization
interquantile_range_32 = [120, 0.27, 0.27]
interquantile_range_16 = [166, 0.24, 0.24]
interquantile_range_8  = [219, 0.20, 0.20]
    
if  NCONSTIT == 8:
    X_train     = X_train / interquantile_range_8
    X_val       = X_val   / interquantile_range_8
    X_test      = X_test  / interquantile_range_8
elif NCONSTIT == 16:
    X_train     = X_train / interquantile_range_16
    X_val       = X_val   / interquantile_range_16
    X_test      = X_test  / interquantile_range_16
elif NCONSTIT == 32:
    X_train     = X_train / interquantile_range_32
    X_val       = X_val   / interquantile_range_32
    X_test      = X_test  / interquantile_range_32
    
    
# Flatten data for MLP input
NINPUT = NCONSTIT * NFEAT
X_train = X_train.reshape(-1, NINPUT)
X_val  = X_val.reshape(-1, NINPUT)
X_test  = X_test.reshape(-1, NINPUT)
    
    
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)



# Set Optuna
study = optuna.create_study(
        study_name="mlp_study",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.HyperbandPruner(),
        direction="maximize",
        storage="sqlite:///optuna_mlp_{}_constit.db".format(NCONSTIT),
        load_if_exists=True,
    )
        
    
    
# Optimize
#study.optimize(objective(data), n_trials=15, gc_after_trial=True)
study.optimize(objective, n_trials=NTRIALS, gc_after_trial=True)


# Results
#trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
trials = study.get_trials(deepcopy=False)
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of trials: ", len(trials))

# 
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

    
# reload study and continue optimizing
#study = optuna.load_study(study_name="example-study", storage="sqlite:///example.db")
#study.optimize(objective, n_trials=30)    



GPU: METAL
Activating procedural GPU memory growth...
kfold numbet of folds: 5
val fold num: 4

----------------
Data loading complete:
File name: None
Training data size: 480,840
Test data size: 120,210
Number of constituents: 32
Number of features: 3
----------------

(480840, 32, 3)
(480840, 96) (120210, 96) (480840, 5) (120210, 5)


[I 2023-11-03 07:26:03,012] Using an existing study with name 'mlp_study' instead of creating a new one.
2023-11-03 07:26:03.207554: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3757/3757 - 1s - loss: 1.2525 - categorical_accuracy: 0.4745 - 791ms/epoch - 211us/step


[I 2023-11-03 07:27:21,509] Trial 2 finished with value: 0.47454455494880676 and parameters: {'nlayers': 1, 'nhidden_l0': 42, 'learning_rate': 0.00011802120724693812, 'bsize': 128}. Best is trial 2 with value: 0.47454455494880676.


3757/3757 - 1s - loss: 1.0213 - categorical_accuracy: 0.5857 - 820ms/epoch - 218us/step


...

3757/3757 - 1s - loss: 0.9169 - categorical_accuracy: 0.6389 - 1s/epoch - 274us/step


[I 2023-11-03 10:42:06,177] Trial 50 finished with value: 0.6389152407646179 and parameters: {'nlayers': 3, 'nhidden_l0': 243, 'nhidden_l1': 13, 'nhidden_l2': 96, 'learning_rate': 0.00021594626953928036, 'bsize': 32}. Best is trial 48 with value: 0.6465518474578857.


3757/3757 - 1s - loss: 0.9193 - categorical_accuracy: 0.6378 - 947ms/epoch - 252us/step


[I 2023-11-03 10:48:49,628] Trial 51 finished with value: 0.6378005146980286 and parameters: {'nlayers': 3, 'nhidden_l0': 178, 'nhidden_l1': 10, 'nhidden_l2': 107, 'learning_rate': 0.00023883917025148635, 'bsize': 32}. Best is trial 48 with value: 0.6465518474578857.


Study statistics: 
  Number of finished trials:  52
  Number of trials:  52
Best trial:
  Value:  0.6465518474578857
  Params: 
    nlayers: 3
    nhidden_l0: 246
    nhidden_l1: 14
    nhidden_l2: 125
    learning_rate: 0.0002518045355374134
    bsize: 32


## MLP 32:
## BEST  Trial 48 finished with value: 0.6465518474578857 and parameters: {'nlayers': 3, 'nhidden_l0': 246, 'nhidden_l1': 14, 'nhidden_l2': 125, 'learning_rate': 0.0002518045355374134, 'bsize': 32}

## MLP 16 : 
##  BEST Trial 48 finished with value:0.655561089515686 and parameters: {'bsize': 64, 'nlayers': 3, 'nhidden_l0': 171, 'nhidden_l1': 30, 'nhidden_l2': 153, 'learning_rate': 0.00025664296269595563}.

## MLP 8:
## BEST Trial 47 finished with value: 0.6322103142738342 and parameters: {'bsize': 64, 'nlayers': 3, 'nhidden_l0': 152, 'nhidden_l1': 69, 'nhidden_l2': 63, 'learning_rate': 0.00023812077479493001}